## 1. Orchestra Celebration
<p>The New York Philharmonic is one of America's largest orchestras. In honor of its many famous musicians, the director is planning a special event. She wants to hold a concert to celebrate the top soloists from its history. You have been asked to determine which soloists should receive recognition.</p>
<p>The director has given you the following requirements:</p>
<ul>
<li><p>The results should only include the top 1% of soloists calculated by the total number of concerts performed.</p></li>
<li><p>Limit your results to entries where the orchestra is 'New York Philharmonic' and the event type is 'Subscription Season'. Soloist performances in other orchestras or event types should not count towards the calculation of the top soloists.</p></li>
<li><p>The director is only interested in individual soloists. You will need to exclude all of the following soloists: 'Chorus', 'No Soloist', 'New York Choral Artists', and 'Schola Cantorum of NY'. You should also exclude any soloist with 'choir' in the name.</p></li>
</ul>
<p>Your result should contain the following columns. It should meet all requirements as described.</p>
<table>
<thead>
<tr>
<th>column</th>
<th>requirements</th>
</tr>
</thead>
<tbody>
<tr>
<td>name</td>
<td>The name of the soloist, with the first name followed by the last name (e.g., Jane Smith). Please note that in the soloists table, names are in reverse order (last name, first name).</td>
</tr>
<tr>
<td>first_date</td>
<td>The first date the soloist ever performed with the orchestra, in the format '01 Jan 2015' (i.e., day as an integer, short month name, and year as an integer).</td>
</tr>
<tr>
<td>last_date</td>
<td>The last date the soloist ever performed with the orchestra, in the format '01 Jan 2015' (i.e., day as an integer, short month name, and year as an integer).</td>
</tr>
<tr>
<td>total_concerts</td>
<td>The total number of concerts the soloist performed.</td>
</tr>
</tbody>
</table>
<p>Order your results by the total number of concerts performed in descending order, and then by soloist name in alphabetical order. The data you will need is available in the two tables detailed in the schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5979/datasets/cad99976082fa05e47621975dc02b4b39c8400ba/symph3.PNG" alt="Database Schema" width="400px"></p>

In [71]:
%%sql
postgresql:///orchestra

WITH main_query as    
(SELECT
     trim(concat(substring(soloist_name from position(',' in soloist_name)+1 for length(soloist_name)),' ',
                substring(soloist_name from 0 for position(',' in soloist_name)))) AS name,
     to_char(min(c.date)::date, 'DD Mon YYYY') AS first_date,
     to_char(max(c.date)::date, 'DD Mon YYYY') AS last_date,
     count( concertnumber) AS total_concerts
 
FROM concerts as c right join soloists  as s
        on s.id = c.id and s.program_id = c.programid 
WHERE 
    orchestra = 'New York Philharmonic' 
    and eventtype = 'Subscription Season'
    and soloist_name NOT IN ('Chorus', 
                             'No Soloist', 
                             'New York Choral Artists', 
                             'Schola Cantorum of NY')
    and lower(soloist_name) NOT LIKE ('%choir%')
  
GROUP BY s.soloist_name)

SELECT  name, 
        first_date, 
        last_date, 
        total_concerts
FROM main_query
ORDER BY
    total_concerts DESC,
    name ASC
LIMIT(
    SELECT(
        COUNT(name)* 0.01)::integer
        FROM(SELECT
                s.soloist_name AS name,
                COUNT(s.id) AS total_concerts
             FROM
                soloists s
             JOIN
                concerts c
             ON
                s.id = c.id
             WHERE
                c.orchestra = 'New York Philharmonic'
             AND
                c.eventtype = 'Subscription Season'
             AND
                s.soloist_name NOT IN ('Chorus','No Soloist','New York Choral Artists','Schola Cantorum of NY')
             AND
                LOWER(s.soloist_name) NOT LIKE '%choir%'
             GROUP BY
                s.soloist_name) AS s);

29 rows affected.


name,first_date,last_date,total_concerts
Glenn Dicterow,16 Oct 1980,25 Jan 2014,277
Philip Smith,08 Nov 1979,17 Dec 2011,212
Philip Myers,24 Jan 1980,07 Nov 2015,193
Stanley Drucker,12 Oct 1961,09 Jun 2009,186
John Corigliano,17 Dec 1921,18 Apr 1966,172
Lorne Munroe,22 Oct 1964,03 Feb 1996,145
Carter Brey,27 Nov 1996,08 Jun 2017,129
Thomas Stacy,19 Apr 1973,14 Mar 2009,129
The Camerata Singers,09 Apr 1964,14 Sep 1977,123
Joseph Robinson,30 Nov 1978,25 Jan 2003,119


%%sql
postgresql:///orchestra
    
SELECT
    CONCAT(TRIM(BOTH FROM SUBSTRING(s.soloist_name, STRPOS(s.soloist_name,',') + 2)),' ',TRIM(BOTH FROM SUBSTRING(s.soloist_name,0,STRPOS(s.soloist_name,',')))) AS name,
    TO_CHAR(MIN(c.date)::date,'DD Mon YYYY') AS first_date,
    TO_CHAR(MAX(c.date)::date,'DD Mon YYYY') AS last_date,
    COUNT(s.id) AS total_concerts
FROM
    soloists s
JOIN
    concerts c
ON
    s.id = c.id
WHERE
    c.orchestra = 'New York Philharmonic'
AND
    c.eventtype = 'Subscription Season'
AND
    s.soloist_name NOT IN ('Chorus','No Soloist','New York Choral Artists','Schola Cantorum of NY')
AND
    LOWER(s.soloist_name) NOT LIKE '%choir%'
GROUP BY
    s.soloist_name
ORDER BY
    total_concerts DESC,
    name ASC
LIMIT(
    SELECT(
        COUNT(name)* 0.01)::integer
        FROM(SELECT
                s.soloist_name AS name,
                COUNT(s.id) AS total_concerts
             FROM
                soloists s
             JOIN
                concerts c
             ON
                s.id = c.id
             WHERE
                c.orchestra = 'New York Philharmonic'
             AND
                c.eventtype = 'Subscription Season'
             AND
                s.soloist_name NOT IN ('Chorus','No Soloist','New York Choral Artists','Schola Cantorum of NY')
             AND
                LOWER(s.soloist_name) NOT LIKE '%choir%'
             GROUP BY
                s.soloist_name) AS s);

%%sql
postgresql:///orchestra
    
    
    


WITH comma AS(
    SELECT 
        id,
        soloist_instrument,
        soloist_roles,
        soloist_name, 
        POSITION(',' IN soloist_name) AS a_comma
    FROM soloists 
    WHERE soloist_instrument NOT IN ('Chorus', 'No Soloist', 'New York Choral Artists', 'Schola Cantorum of NY')
    AND soloist_roles IS NOT NULL
    GROUP BY entry_number)

SELECT
    DISTINCT(ltrim(rtrim(substring(soloist_name,a_comma+1,100))) || ' ' || ltrim(rtrim(LEFT(soloist_name,a_comma-1)))) as name,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date ASC),'YYYY MM DD'), 'DD Mon YYYY')  AS first_date,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date DESC), 'YYYY MM DD'), 'DD Mon YYYY') AS last_date,
    COUNT(c.programid) OVER(PARTITION BY c.programid) AS total_concerts
FROM concerts as c
INNER JOIN comma as s
ON c.id = s.id
WHERE c.orchestra ='New York Philharmonic' AND c.eventtype = 'Subscription Season'
ORDER BY total_concerts desc, name ASC
LIMIT 93


    
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql:///orchestra')
print(engine.table_names())


df_soloist_query = """
SELECT * FROM soloists
"""

solo_df = pd.read_sql(df_soloist_query, engine)

"""
conc : id, orchestra, eventtype, date
selected_orcestra = 'New York Philharmonic'
selected_eventtype = 'Subscription Season'

solo : id, soloist_instrument, soloist_name
excluded_solo_instrument = (Chorus', 'No Soloist', 'New York Choral Artists', and 'Schola Cantorum of NY')
excluded_name = 'choir'
should_drop soloist_role NaN
"""
main_query = """

WITH comma AS(
    SELECT 
        id,
        soloist_instrument,
        soloist_roles,
        soloist_name, 
        POSITION(',' IN soloist_name) AS a_comma
    FROM soloists 
    WHERE soloist_instrument NOT IN ('Chorus', 'No Soloist', 'New York Choral Artists', 'Schola Cantorum of NY')
    AND soloist_roles IS NOT NULL
    GROUP BY entry_number)

SELECT
    DISTINCT(ltrim(rtrim(substring(soloist_name,a_comma+1,100))) || ' ' || ltrim(rtrim(LEFT(soloist_name,a_comma-1)))) as name,
    s.id,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date ASC),'YYYY MM DD'), 'DD Mon YYYY')  AS first_date,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date DESC), 'YYYY MM DD'), 'DD Mon YYYY') AS last_date,
    COUNT(c.programid) OVER(PARTITION BY c.programid) AS total_concerts,
    PERCENT_RANK()
FROM concerts as c
INNER JOIN comma as s
ON c.id = s.id
WHERE c.orchestra ='New York Philharmonic' AND c.eventtype = 'Subscription Season'
ORDER BY total_concerts desc, name ASC
"""
main_df = pd.read_sql(main_query,engine)

exclude_instrument_try = """
SELECT 
    id,
    soloist_instrument,
    soloist_roles,
    soloist_name
FROM soloists 
WHERE soloist_instrument NOT IN ('Chorus', 'No Soloist', 'New York Choral Artists', 'Schola Cantorum of NY')
"""

exclude_instrument_df = pd.read_sql(exclude_instrument_try, engine)

tryname_query = """
WITH comma AS(
SELECT 
    soloist_name,
    soloist_roles,
    POSITION(',' IN soloist_name) AS a_comma
FROM soloists
)
SELECT
     ltrim(rtrim(substring(soloist_name,a_comma+1,100))) || ' ' || ltrim(rtrim(LEFT(soloist_name,a_comma-1))) AS name,
     soloist_roles
FROM comma
"""

trycomma_df = pd.read_sql(tryname_query, engine)

try_first_value = """
SELECT
    to_char(to_date(FIRST_VALUE(date) OVER(PARTITION BY id ORDER BY date), 'YYYY MM DD'), 'DD Mon YYYY') AS first_date
FROM concerts

"""
try_date_df = pd.read_sql(try_first_value, engine)

"""
conc : id, orchestra, eventtype, date
selected_orcestra = 'New York Philharmonic'
selected_eventtype = 'Subscription Season'

solo : id, soloist_instrument, soloist_name
excluded_solo_instrument = (Chorus', 'No Soloist', 'New York Choral Artists', and 'Schola Cantorum of NY')
excluded_name = 'choir'
should_drop soloist_role NaN
"""
main_query = """

WITH comma AS(
SELECT
    soloist_name, 
    POSITION(',' IN soloist_name) AS a_comma
FROM soloists)

SELECT
    (SELECT ltrim(rtrim(substring(soloist_name,a_comma+1,100))) || ' ' || ltrim(rtrim(LEFT(soloist_name,a_comma-1))) as name
     FROM comma),
    s.id,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date ASC),'YYYY MM DD'), 'DD Mon YYYY')  AS first_date,
    to_char(to_date(FIRST_VALUE(c.date)
    OVER(PARTITION BY c.id ORDER BY date DESC), 'YYYY MM DD'), 'DD Mon YYYY') AS last_date,
    COUNT(c.programid) OVER(PARTITION BY c.programid) AS total_concerts
FROM concerts as c
INNER JOIN (
    SELECT 
        id,
        soloist_instrument,
        soloist_roles
    FROM soloists 
    WHERE soloist_instrument NOT IN ('Chorus', 'No Soloist', 'New York Choral Artists', 'Schola Cantorum of NY')
    AND soloist_roles IS NOT NULL
    GROUP BY entry_number
    ) as s
ON c.id = s.id
WHERE c.orchestra ='New York Philharmonic' AND c.eventtype = 'Subscription Season'
ORDER BY total_concerts desc, name ASC
"""
main_df = pd.read_sql(main_query,engine)